In [81]:
import pandas as pd
df = pd.read_csv("../../data/property_data/2023_property_data.csv")
df.head()

,Unnamed: 0,district_code,property_id,sale_counter,download_datetime,property_name,unit_number,house_number,street_name,locality,...,settlement_date,purchase_price,zoning,nature_of_property,primary_purpose,strata_lot_number,component_code,sale_code,interest_percentage,dealing_number
0,0,1,4246333.0,1,2023-03-13 01:05:00,NaN,NaN,36,WINE COUNTRY DR,CESSNOCK,...,2023-03-03,6480000,RU2,3,COMMERCIAL,NaN,RAL,NaN,0.0,AS900212
1,1,1,4172373.0,2,2023-03-13 01:05:00,NaN,NaN,14,CAMPBELL ST,ELLALONG,...,2023-03-02,780000,RU5,R,RESIDENCE,NaN,RAB,NaN,0.0,AS896657
2,2,1,4140930.0,3,2023-03-13 01:05:00,NaN,1,3,ALLMAN ST,CLIFTLEIGH,...,2023-03-06,810000,R2,R,RESIDENCE,NaN,AAP,NaN,0.0,AS904194
3,3,1,350.0,4,2023-03-13 01:05:00,NaN,NaN,27,COLLIERY ST,ABERDARE,...,2023-03-07,670000,R2,R,RESIDENCE,NaN,AAD,NaN,0.0,AS907683
4,4,1,8143.0,5,2023-03-13 01:05:00,NaN,NaN,70,MOUNT VIEW RD,CESSNOCK,...,2023-03-03,715000,R2,R,RESIDENCE,NaN,AAB,NaN,0.0,AS901129


In [103]:
import pandas as pd

def post_2001_format(year, df):
    # Select required columns
    df = df[["post_code", "area", "contract_date", "settlement_date", "purchase_price", "nature_of_property", "primary_purpose"]]
    
    # Drop rows with missing dates
    df = df[df["settlement_date"].notna() & df["contract_date"].notna()]
    
    # Convert settlement_date and contract_date to datetime with coercion to handle invalid dates
    df["settlement_date"] = pd.to_datetime(df["settlement_date"], errors='coerce', dayfirst=True)
    df["contract_date"] = pd.to_datetime(df["contract_date"], errors='coerce', dayfirst=True)
    
    # Drop rows where date conversion failed (NaT values)
    df = df.dropna(subset=["settlement_date", "contract_date"])
    
    # Update year for settlement_date and contract_date to match the given year
    df["settlement_date"] = df["settlement_date"].apply(lambda x: x.replace(year=year) if x.month != 2 or x.day <= 28 else pd.NaT)
    df["contract_date"] = df["contract_date"].apply(lambda x: x.replace(year=year) if x.month != 2 or x.day <= 28 else pd.NaT)
    
    # Drop rows with NaT created from invalid date replacements
    df = df.dropna(subset=["settlement_date", "contract_date"])
    
    # Reformat dates to 'dd/mm/yyyy'
    df["settlement_date"] = df["settlement_date"].dt.strftime('%d/%m/%Y')
    df["contract_date"] = df["contract_date"].dt.strftime('%d/%m/%Y')
    
    # Add year column and filter by purchase price
    df["year"] = year
    df = df[(df["purchase_price"].notna()) & (df["purchase_price"] < 100000000) & (df["purchase_price"] > 100000)]
    df=df[df["post_code"].notna()]
    nsw_postcodes = list(range(2000, 2235)) + list(range(2250, 2999)) + list(range(2619, 2900))
    df["post_code"]=df["post_code"].astype("int32")
    df = df[df["post_code"].isin(nsw_postcodes)]
    return df


In [107]:
post_2001_data=[]
def clean_post_2001_data():
    years = range(2002,2024)
    for year in years:
        path = f"../../data/property_data/{year}_property_data.csv"
        df=pd.read_csv(path)
        df_final = post_2001_format(year,df)
        df_final.to_csv(f"../../data/clean_data/{year}_property_data.csv",index=False)
        post_2001_data.append(df_final)
    full_df = pd.concat(post_2001_data,ignore_index=True)
    full_df.to_csv(f"../../data/database_files/post_2021_data.csv",index=False)

In [108]:
clean_post_2001_data()

/var/folders/74/9tpvyg253_d1_06tsh3c83900000gn/T/ipykernel_41827/2032411526.py:6: DtypeWarning: Columns (6,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path)
/var/folders/74/9tpvyg253_d1_06tsh3c83900000gn/T/ipykernel_41827/3229628448.py:11: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["settlement_date"] = pd.to_datetime(df["settlement_date"], errors='coerce', dayfirst=True)
/var/folders/74/9tpvyg253_d1_06tsh3c83900000gn/T/ipykernel_41827/3229628448.py:12: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["contract_date"] = pd.to_datetime(df["contract_date"], errors='coerce', dayfirst=True)
/var/folders/74/9tpvyg253_d1_06tsh3c83900000gn/T/ipykernel_41827/3229628448.py:11: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=T

In [114]:
def pre_2002_format(year, df):
    # Select required columns
    df = df[["postcode", "area", "contract_date",  "purchase_price"]]
    
    # Drop rows with missing dates
    df = df[df["contract_date"].notna()]
    
    
    df["contract_date"] = pd.to_datetime(df["contract_date"], errors='coerce', dayfirst=True)
    
    # Drop rows where date conversion failed (NaT values)
    df = df.dropna(subset=["contract_date"])
    
    # Update year for settlement_date and contract_date to match the given year
    
    df["contract_date"] = df["contract_date"].apply(lambda x: x.replace(year=year) if x.month != 2 or x.day <= 28 else pd.NaT)
    
    # Drop rows with NaT created from invalid date replacements
    df = df.dropna(subset=["contract_date"])
    
    # Reformat dates to 'dd/mm/yyyy'
    
    df["contract_date"] = df["contract_date"].dt.strftime('%d/%m/%Y')
    
    # Add year column and filter by purchase price
    df["year"] = year
    df = df[(df["purchase_price"].notna()) & (df["purchase_price"] < 100000000) & (df["purchase_price"] > 100000)]
    df=df[df["postcode"].notna()]
    nsw_postcodes = list(range(2000, 2235)) + list(range(2250, 2999)) + list(range(2619, 2900))
    df["post_code"]=df["postcode"].astype("int32")
    df = df[df["post_code"].isin(nsw_postcodes)]
    df.drop(['postcode'],axis=1,inplace=True)
    return df

In [117]:
pre_2002_data=[]
def clean_pre_2002_data():
    years = range(1990,2002)
    for year in years:
        path = f"../../data/property_data/{year}_property_data.csv"
        df=pd.read_csv(path)
        df_final = pre_2002_format(year,df)
        df_final.to_csv(f"../../data/clean_data/{year}_property_data.csv",index=False)
        pre_2002_data.append(df_final)
    full_df = pd.concat(pre_2002_data,ignore_index=True)
    full_df.to_csv(f"../../data/database_files/pre_2002_data.csv",index=False)

In [118]:
clean_pre_2002_data()

/var/folders/74/9tpvyg253_d1_06tsh3c83900000gn/T/ipykernel_41827/812146354.py:9: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["contract_date"] = pd.to_datetime(df["contract_date"], errors='coerce', dayfirst=True)
/var/folders/74/9tpvyg253_d1_06tsh3c83900000gn/T/ipykernel_41827/812146354.py:9: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["contract_date"] = pd.to_datetime(df["contract_date"], errors='coerce', dayfirst=True)
/var/folders/74/9tpvyg253_d1_06tsh3c83900000gn/T/ipykernel_41827/812146354.py:9: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["contract_date"] = pd.to_datetime(df["contract_date"], errors='coerce', dayfirst=True)
/var/folders/74/9tpvyg253_d1_06tsh3c8390